In [2]:
import numpy as np
import networkx as nx
import data,network,torch,pickle
import matplotlib.pyplot as plt
import torch_geometric.utils as utils
import torch.nn.functional as F
from torch_geometric.utils import to_networkx
from torch_geometric.loader import DataLoader as batchLoader

device = 'cpu'
for modelIdx in ["tp"]:#range(33,49):

    path = './trainedModel/'
    metaPath = f'{modelIdx}_metadata.pkl'
    modelPath = f'{modelIdx}.pt'
    paraPath = f'{modelIdx}_para.pkl'

    with open(path+metaPath,'rb') as f: metadata = pickle.load(f)
    with open(path+paraPath,'rb') as f: modelPara,paraDict = pickle.load(f)
    paraDict['device'] = torch.device(device)
    checkPoint = torch.load(path+modelPath,map_location=torch.device(device))
    model = network.modelSelect(modelPara).to(device)
    model.load_state_dict(checkPoint['model_state_dict'])
    model.eval()
    print("Model loaded.")

    FTG = data.FTGenerator(paraDict,metadata)
    Dataset,metadata = FTG.createDataset()
    _,location = FTG._constructRawData()
    testSet = Dataset[-730:]
    # testSet = [Dataset[-1]]
    nClass = metadata['nClass']
    nNodes = testSet[0].x.shape[0]
    print(f"Model {modelIdx}. Test dataset loaded.")

    testLoader = batchLoader(testSet,batch_size=len(testSet),shuffle=False)

    for batch in testLoader:
        _yHat,_weights = model(batch.x,batch.edge_index,batch.edge_attr)
        # Pick the last three days only
        _yHat = _yHat[:,-6:]  
        _truth = batch.y[:,-3:].reshape(-1) 
        _yHat = _yHat.reshape(-1,nClass)
        _pred = torch.argmax(_yHat,axis=-1)
        _correct = _pred == _truth
            
        _accu = int(_correct.sum())/len(_truth)

        _tP = (_truth==1).reshape(-1,nNodes)
        _tN = (_truth==0).reshape(-1,nNodes)
        _pP = (_pred==1).reshape(-1,nNodes)
        _pN = (_pred==0).reshape(-1,nNodes)
        
        _TP = torch.sum(_tP & _pP,axis=0)
        _FN = torch.sum(_tP & _pN,axis=0)
        _FP = torch.sum(_tN & _pP,axis=0)
        _TN = torch.sum(_tN & _pN,axis=0)

        _recall = _TP/(_TP+_FN)
        _prec   = _TP/(_TP+_FP)
        _f1     = 2*_TP/(2*_TP+_FP+_FN)
        print(_accu,_recall.mean().item(),_prec.mean().item(),_f1.mean().item())
        with open('record2','a') as f:
            print(_accu,_recall.mean().item(),_prec.mean().item(),_f1.mean().item(),file=f)

Model loaded.
Model tp. Test dataset loaded.
0.9388278388278388 0.596262514591217 0.6688671708106995 0.6299761533737183


In [ ]:
import numpy as np
import networkx as nx
import data,network,torch,pickle
#import matplotlib.pyplot as plt
#import torch_geometric.utils as utils
#import torch.nn.functional as F
#from torch_geometric.utils import to_networkx
#from torch_geometric.loader import DataLoader as batchLoader


/home/julian/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from torch_geometric.utils import to_dense_adj

In [ ]:
import numpy as np
from glob import glob
from scipy.io import loadmat
import torch
from sklearn import neighbors as nb
import torch_geometric.utils as utils
#from torch_geometric.data import Data
#mport pandas as pd


/home/julian/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
